In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from geopy.distance import geodesic
import numpy as np

# Folder containing the .csv files
folder_path = "./time_sets"

# Map tile size in nautical miles
tile_width_nm = 5
tile_height_nm = 5

# Conversion from nautical miles to degrees (approximate, depends on latitude)
nm_to_deg = 1 / 60

# Initialize a dictionary to count navigational statuses
nav_status_counts = {}

# Function to calculate bounding box
def calculate_bounding_box(center_lat, center_lon, width_nm, height_nm):
    lat_delta = height_nm * nm_to_deg
    lon_delta = width_nm * nm_to_deg / np.cos(np.radians(center_lat))
    return (center_lat - lat_delta, center_lat + lat_delta,
            center_lon - lon_delta, center_lon + lon_delta)

# Function to plot vessel movements on a map tile
def plot_map(data, file_name, bbox):
    plt.figure(figsize=(8, 8))
    plt.plot(data['Longitude'], data['Latitude'], marker='o', markersize=4, label='Vessel Movement')
    plt.xlim([bbox[2], bbox[3]])  # Set longitude bounds
    plt.ylim([bbox[0], bbox[1]])  # Set latitude bounds
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.title(f"Vessel Movement: {file_name}")
    plt.legend()
    plt.grid()
    plt.savefig(f"map_tiles/{file_name}.png")
    plt.close()

# Create output folder for map tiles
os.makedirs("map_tiles", exist_ok=True)

# Process each file
for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        file_path = os.path.join(folder_path, file)
        
        # Load CSV file
        data = pd.read_csv(file_path)
        
        # Ensure 'Navigational status' has a single unique value
        unique_status = data['Navigational status'].unique()
        if len(unique_status) == 1:
            status = unique_status[0]
            nav_status_counts[status] = nav_status_counts.get(status, 0) + 1
            
            # Calculate the bounding box for the map tile
            center_lat = data['Latitude'].mean()
            center_lon = data['Longitude'].mean()
            bbox = calculate_bounding_box(center_lat, center_lon, tile_width_nm, tile_height_nm)
            
            # Plot the map tile
            plot_map(data, file.replace(".csv", ""), bbox)

# Display the counts of navigational statuses
print("Counts of Navigational Statuses:")
for status, count in nav_status_counts.items():
    print(f"Status {status}: {count} files")


: 